In [1]:
pip install pytube


     ---------------------------------------- 57.6/57.6 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from pytube import Playlist, YouTube
from tqdm import tqdm
import os
import requests
import re

def sanitize_filename(filename):
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

def download_playlist_videos(playlists, output_folder):
    for playlist_url in playlists:
        try:
            # Create a Playlist object
            playlist = Playlist(playlist_url)

            # Get the name of the playlist to create a subdirectory inside the output folder
            playlist_name = playlist.title
            playlist_name = sanitize_filename(playlist_name)  # Sanitize the playlist title

            # Create a directory inside the output folder with the name of the sanitized playlist
            playlist_dir = os.path.join(output_folder, playlist_name)
            os.makedirs(playlist_dir, exist_ok=True)

            # Function to download video and show progress
            def download_with_progress(video, output_path, index):
                stream = video.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
                total_size = stream.filesize
                bytes_downloaded = 0

                with requests.get(stream.url, stream=True) as response:
                    response.raise_for_status()
                    with open(output_path, 'wb') as output_file:
                        with tqdm(total=total_size, unit='B', unit_scale=True, 
                                  desc=f"{index:02d} - {video.title} ({stream.resolution})", ncols=80) as pbar:
                            for chunk in response.iter_content(chunk_size=8192):
                                output_file.write(chunk)
                                bytes_downloaded += len(chunk)
                                pbar.update(len(chunk))

            # Download each video in the playlist
            for index, video_url in enumerate(playlist.video_urls, start=1):
                # Create a Video object
                video = YouTube(video_url)

                # Print the title of the video being downloaded
                print(f"Downloading {index:02d} - {video.title}")

                # Prepare the output path for the video
                video_filename = video.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().default_filename
                output_path = os.path.join(playlist_dir, f"{index:02d} - {sanitize_filename(video_filename)}")

                # Download the video with progress tracking
                download_with_progress(video, output_path, index)

                # Print a message after successful download
                print(f"Download {index:02d} - {video.title} complete.")

        except Exception as e:
            print(f"Error downloading {playlist_url}: {str(e)}")

# Example usage:
if __name__ == "__main__":
    playlists = [
        "https://www.youtube.com/playlist?list=PL-Jc9J83PIiFkOETg2Ybq-FMuJjkZSGeH",
        "https://www.youtube.com/playlist?list=PL-Jc9J83PIiHq5rMZasunIR19QG3E-PAA"
        # Add more playlist URLs if needed
    ]

    output_folder = "output"
    download_playlist_videos(playlists, output_folder)


01 - Introduction to 2-D Arrays | Data Structures and Algorithms in JAVA (720p):